# Exercice 1 - PointMLP (ModelNet40_PLY)

## 1) Architecture check against the PDF instructions

| Requirement from Exercise 1 | In `Code/pointnet.py` | Status |
|---|---|---|
| Flatten point cloud `1024 x 3 -> 3072` | `x = input.reshape(input.size(0), -1)` in `PointMLP.forward` | OK |
| First layer `MLP(3072, 512)` | `self.fc1 = nn.Linear(3072, 512)` | OK |
| Second layer `MLP(512, 256)` | `self.fc2 = nn.Linear(512, 256)` | OK |
| Dropout `p = 0.3` on second stage | `self.dropout = nn.Dropout(0.3)` | OK |
| Last layer `MLP(256, N_classes)` | `self.fc3 = nn.Linear(256, classes)` | OK |
| BatchNorm + ReLU on hidden layers | `bn1/bn2` + `F.relu` after `fc1/fc2` | OK |
| `LogSoftmax` output for class scores | `self.log_softmax = nn.LogSoftmax(dim=1)` | OK |

Note: there is no BatchNorm/ReLU after the final classification layer, which is the standard design before `LogSoftmax`.

## 2) PointMLP architecture used

![architecture MLP](architecture_mlp.png)

## 3) Training curves analysis (250 epochs)

![training curves](training_curves.png)

- Loss decreases quickly in the first 20-40 epochs, then reaches a plateau.
- End of training from the curves: train loss ~2.45, test loss ~2.55.
- Accuracy also saturates early (around epoch 60-80).
- End of training from the curves: train accuracy ~27-28%, test accuracy ~22-23%.
- Generalization gap is around 5 points, but both accuracies stay low: this indicates limited representational power for this task.

## 4) Final metrics measured on the saved model (`pointmlp_modelnet40.pth`)

Evaluation done with deterministic preprocessing (no random rotation/noise/shuffle during evaluation):

| Split | N samples | NLL loss | Accuracy |
|---|---:|---:|---:|
| Train | 9843 | 2.3570 | 29.27% |
| Test | 2468 | 2.5317 | 22.97% |

For reference, random guessing on ModelNet40 is `1/40 = 2.5%`, so the model learns something but remains far from good classification performance.

## 4bis) Evaluation protocol note

In the current training script (`Code/pointnet.py`), `test_ds` is created with default transforms, which include random rotation/noise/shuffle. For strict benchmarking, test preprocessing should usually be deterministic (`ToTensor` only).
This is why we report both: (1) the training-curve trend, and (2) deterministic final metrics from the saved model.

## 5) Answer to Exercise 1 questions

1. **Test accuracy (ModelNet40_PLY, PointMLP):** **22.97%**.
2. **Comment:** the network underfits. A fully connected MLP on flattened coordinates does not model geometric structure well (local neighborhoods, shape composition, permutation invariance), so performance saturates quickly at a low level.

## 6) Why results are limited

- Flattening destroys explicit 3D locality.
- The model is sensitive to point ordering, while point clouds are unordered sets.
- With random point shuffling, the same shape can appear in different flattened orders, which is especially difficult for a plain MLP.
